In [32]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

df = pd.read_csv("../../data/processed/titanic3_eda.csv")

target = "survived"
features = ["pclass", "sex", "age", "fare", "sibsp", "parch", "embarked", "is_child"]

X = df[features].copy()
y = df[target].astype(np.float32).copy()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train survival rate:", y_train.mean(), "Test survival rate:", y_test.mean())


Train: (1047, 8) Test: (262, 8)
Train survival rate: 0.38204393 Test survival rate: 0.3816794


In [33]:
num_cols = ["pclass", "age", "fare", "sibsp", "parch", "is_child"]
cat_cols = ["sex", "embarked"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

# Fit on TRAIN ONLY, then transform train/test
X_train_np = preprocess.fit_transform(X_train)
X_test_np  = preprocess.transform(X_test)

# Convert to dense if needed (sometimes it's sparse)
X_train_np = X_train_np.toarray() if hasattr(X_train_np, "toarray") else X_train_np
X_test_np  = X_test_np.toarray() if hasattr(X_test_np, "toarray") else X_test_np

print("After preprocessing:", X_train_np.shape, X_test_np.shape)



After preprocessing: (1047, 11) (262, 11)


In [34]:
X_train_t = torch.tensor(X_train_np, dtype=torch.float32)
X_test_t  = torch.tensor(X_test_np, dtype=torch.float32)

y_train_t = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_t  = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds  = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=256, shuffle=False)


In [35]:
class MLP(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)   # logits (raw scores), NOT probabilities
        )

    def forward(self, x):
        return self.net(x)

model = MLP(in_dim=X_train_t.shape[1])
model


MLP(
  (net): Sequential(
    (0): Linear(in_features=11, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [36]:
loss_fn = nn.BCEWithLogitsLoss()   # combines sigmoid + BCE in a stable way
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def accuracy_from_logits(logits, y_true):
    probs = torch.sigmoid(logits)
    preds = (probs >= 0.5).float()
    return (preds == y_true).float().mean().item()

for epoch in range(0, 100):
    model.train()
    train_losses = []
    train_accs = []

    for xb, yb in train_loader:
        logits = model(xb)
        loss = loss_fn(logits, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_accs.append(accuracy_from_logits(logits.detach(), yb))

    model.eval()
    with torch.no_grad():
        test_logits = model(X_test_t)
        test_loss = loss_fn(test_logits, y_test_t).item()
        test_acc = accuracy_from_logits(test_logits, y_test_t)

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={np.mean(train_losses):.4f} train_acc={np.mean(train_accs):.4f} | "
        f"test_loss={test_loss:.4f} test_acc={test_acc:.4f}"
    )


Epoch 00 | train_loss=0.6543 train_acc=0.6425 | test_loss=0.6413 test_acc=0.6794
Epoch 01 | train_loss=0.6308 train_acc=0.6793 | test_loss=0.6179 test_acc=0.7023
Epoch 02 | train_loss=0.6115 train_acc=0.7058 | test_loss=0.5956 test_acc=0.7061
Epoch 03 | train_loss=0.5903 train_acc=0.7190 | test_loss=0.5751 test_acc=0.7137
Epoch 04 | train_loss=0.5673 train_acc=0.7315 | test_loss=0.5566 test_acc=0.7099
Epoch 05 | train_loss=0.5512 train_acc=0.7440 | test_loss=0.5397 test_acc=0.7214
Epoch 06 | train_loss=0.5400 train_acc=0.7534 | test_loss=0.5249 test_acc=0.7328
Epoch 07 | train_loss=0.5254 train_acc=0.7547 | test_loss=0.5118 test_acc=0.7405
Epoch 08 | train_loss=0.5104 train_acc=0.7611 | test_loss=0.5000 test_acc=0.7519
Epoch 09 | train_loss=0.5040 train_acc=0.7650 | test_loss=0.4890 test_acc=0.7786
Epoch 10 | train_loss=0.4933 train_acc=0.7756 | test_loss=0.4784 test_acc=0.7901
Epoch 11 | train_loss=0.4871 train_acc=0.7811 | test_loss=0.4709 test_acc=0.7939
Epoch 12 | train_loss=0.4771